# Module objectives
- Exact lookups
- Semantic search/vector index
- Combine Exact and Semantic search + graph expansion for better context
- The R in GraphRAG (and maybe agents)

In [2]:
!pip install graphdatascience neo4j dotenv openai langchain

Import our usual suspects (and some more...)

In [17]:
import os
import pandas as pd
from dotenv import load_dotenv
from graphdatascience import GraphDataScience
from neo4j import Query, GraphDatabase, RoutingControl, Result
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from openai import OpenAI
from typing import List, Optional
from pydantic import BaseModel, Field, validator
import functools
from langchain_core.tools import tool

# Setup

Load env variables

In [4]:
env_file = 'ws.env'

In [5]:
if os.path.exists(env_file):
    load_dotenv(env_file, override=True)

    # Neo4j
    HOST = os.getenv('NEO4J_URI')
    USERNAME = os.getenv('NEO4J_USERNAME')
    PASSWORD = os.getenv('NEO4J_PASSWORD')
    DATABASE = os.getenv('NEO4J_DATABASE')

    # AI
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
    LLM = os.getenv('LLM')
else:
    print(f"File {env_file} not found.")

Connect to neo4j db

In [6]:
driver = GraphDatabase.driver(
    HOST,
    auth=(USERNAME, PASSWORD)
)
driver.verify_connectivity(database=DATABASE)

/Users/erikbijl/anaconda3/lib/python3.11/site-packages/neo4j/_sync/driver.py:1075: ExperimentalWarning: All configuration key-word arguments to verify_connectivity() are experimental. They might be changed or removed in any future version without prior notice.
  experimental_warn(


## Schema
Keeping this in case we need to add in more indexes

In [7]:
schema_statements = [
    'CREATE CONSTRAINT IF NOT EXISTS FOR (n:Person) REQUIRE (n.email) IS node key',
    'CREATE CONSTRAINT IF NOT EXISTS FOR (n:Skill) REQUIRE (n.name) IS node key',
    'CREATE VECTOR INDEX ada_v IF NOT EXISTS FOR (n:Skill) ON (n.embedding)'
]

In [8]:
for statement in schema_statements:
    driver.execute_query(
        statement,
        database_=DATABASE,
        routing_=RoutingControl.WRITE
    )

Fetch all indexes

In [9]:
schema_result_df  = driver.execute_query(
    'show indexes',
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [10]:
schema_result_df.head(100)

,id,name,state,populationPercent,type,entityType,labelsOrTypes,properties,indexProvider,owningConstraint,lastRead,readCount
0,6,ada_v,ONLINE,100.0,VECTOR,NODE,[Skill],[embedding],vector-2.0,None,None,NaN
1,4,constraint_63bf11a1,ONLINE,100.0,RANGE,NODE,[Skill],[name],range-1.0,constraint_63bf11a1,2025-03-08T08:54:28.113000000+00:00,581.0
2,2,constraint_d3bfd313,ONLINE,100.0,RANGE,NODE,[Person],[email],range-1.0,constraint_d3bfd313,2025-03-08T08:54:28.114000000+00:00,305.0
3,0,index_343aff4e,ONLINE,100.0,LOOKUP,NODE,None,None,token-lookup-1.0,None,2025-03-08T08:54:34.731000000+00:00,57.0
4,1,index_f7700477,ONLINE,100.0,LOOKUP,RELATIONSHIP,None,None,token-lookup-1.0,None,2025-03-08T08:54:02.612000000+00:00,2.0


# Basic search

In [11]:
skills = ['Contineous Delivery', 'Cloud Native', 'Security']

Find person given some skills

In [13]:
search_persons_df = driver.execute_query(
    """
    MATCH (p:Person)-[:KNOWS]->(s:Skill)
    WHERE s.name IN $skills
    RETURN 
        count(*) AS rank, 
        p.email AS email, 
        p.name AS person_name, 
        COLLECT{ MATCH (p)-[:KNOWS]->(anySkill) RETURN anySkill.name } AS skills
    ORDER BY rank DESC LIMIT  10
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df(),
    skills = skills
)

In [14]:
search_persons_df

,rank,email,person_name,skills
0,1,lucy.turner@test.org,Lucy Turner,"[Security, Express.js, Big Data, Scala, Docker]"
1,1,sophie.jackson@test.org,Sophie Jackson,"[Security, Pandas, Linux, Angular]"
2,1,mia.nelson@test.org,Mia Nelson,"[Security, WordPress, Big Data, Swift, AWS]"
3,1,david.lopez@test.org,David Lopez,"[Security, WordPress, PHP]"
4,1,thomas.brown@test.org,Thomas Brown,"[Security, R, Java, Docker]"
5,1,isabella.allen@test.org,Isabella Allen,"[Security, Scala, Cloud Architecture]"
6,1,olivia.johnson@test.org,Olivia Johnson,"[Security, Angular, CI/CD]"
7,1,amelia.davis@test.org,Amelia Davis,"[Security, PyTorch, Java, HTML5, Docker]"
8,1,emily.phillips@test.org,Emily Phillips,"[Security, Vue.js, PHP, Kubernetes, Data Visua..."
9,1,thomas.nelson@test.org,Thomas Nelson,"[Security, Pandas, Go]"


# Vector index search

In [15]:
# client = OpenAI()
# client.models.list()

In [16]:
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
v_skills = embeddings.embed_documents(skills)

### Strategy 1
We get the approximate top 10 nearest nodes to the search vector `v` and take the 3 first returned. Then put them together in a list (`skill_list`) and does same ranking as before (number of skills)

In [ ]:
df_1 = driver.execute_query(
    """
    UNWIND $v_skills AS v
    CALL db.index.vector.queryNodes('ada_v', 10, TOFLOATLIST(v)) YIELD node
    WITH v, COLLECT(node)[0..3] AS top3
    UNWIND top3 AS s
    WITH COLLECT(s) AS skill_list 
    MATCH (p:Person)-[:KNOWS]->(s)
    WHERE s IN skill_list
    RETURN 
        COUNT(*) AS rank, 
        p.email AS email, 
        p.name AS person_name, 
        COLLECT{ MATCH (p)-[:KNOWS]->(anySkill) RETURN anySkill.name } AS skills
    ORDER BY rank DESC 
    LIMIT 10
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df(),
    v_skills = v_skills
)

In [ ]:
df.head(10)

## Strategy 2
Same strategy as before for finding the semantic similar skills, but we do a post sorting based on cosine similarity

In [19]:
df_2 = driver.execute_query(
    """
    UNWIND $v_skills AS v
    CALL db.index.vector.queryNodes('ada_v', 10, TOFLOATLIST(v)) YIELD node
    WITH v, COLLECT(node)[0..3] AS top3
    UNWIND top3 AS s
    WITH COLLECT(s) AS skill_list 
    MATCH (p:Person)-[:KNOWS]->(s)
    WHERE s IN skill_list
    WITH p, SUM(REDUCE(res=0.0, x IN $v_skills | res + vector.similarity.cosine(x, s.embedding))) AS score
    RETURN 
        score AS rank, 
        p.email AS email, 
        p.name AS person_name, 
        COLLECT{ MATCH (p)-[:KNOWS]->(anySkill) RETURN anySkill.name } AS skills
    ORDER BY rank DESC 
    LIMIT 10
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df(),
    v_skills = v_skills
)

In [20]:
df_2.head(10)

,rank,email,person_name,skills
0,8.097379,david.rodriguez@test.org,David Rodriguez,"[Scrum, Azure, Cypher, Spark]"
1,5.481543,isabella.allen@test.org,Isabella Allen,"[Security, Scala, Cloud Architecture]"
2,5.475172,olivia.johnson@test.org,Olivia Johnson,"[Security, Angular, CI/CD]"
3,5.417244,andrew.anderson@test.org,Andrew Anderson,"[DevOps, Java, Testing, Node.js, Data Visualiz..."
4,5.414759,andrew.martin@test.org,Andrew Martin,"[R, Java, Cloud Architecture, Testing, Node.js]"
5,5.411052,sophia.walker@test.org,Sophia Walker,"[Scrum, DevOps, Django, C++]"
6,5.411007,natalie.miller@test.org,Natalie Miller,"[Go, Express.js, Azure, Testing, Machine Learn..."
7,5.408567,sophie.perez@test.org,Sophie Perez,"[Scrum, Angular, Cloud Architecture]"
8,5.408567,isabella.jones@test.org,Isabella Jones,"[Scrum, TensorFlow, Cloud Architecture, ReactJS]"
9,5.408388,joseph.lopez@test.org,Joseph Lopez,"[Linux, System Design, CI/CD, Django, TypeScri..."


## Strategy 3
We can also look at community membership (from our kmeans community detection from before). We could for instance imagining the user wanting to explore the community that looks most relevant.


In [21]:
df_3 = driver.execute_query(
    """
    UNWIND $v_skills AS v
    CALL db.index.vector.queryNodes('ada_v', 10, TOFLOATLIST(v)) YIELD node
    WITH v, COLLECT(node)[0..3] AS top3
    UNWIND top3 AS s
    WITH COLLECT(s) AS skill_list 
    MATCH (p:Person)-[:KNOWS]->(s)
    WHERE s IN skill_list
    WITH p, SUM(REDUCE(res=0.0, x IN $v_skills | res + vector.similarity.cosine(x, s.embedding))) AS score
    WITH
        p.kmeans5_cluster AS community,
        score AS rank, 
        p.email AS email, 
        p.name AS person_name, 
        COLLECT{ MATCH (p)-[:KNOWS]->(anySkill) RETURN anySkill.name ORDER BY anySkill.name } AS skills
    ORDER BY rank DESC 
    LIMIT 10
    RETURN 
        community,
        rank,
        email,
        person_name,
        skills
    ORDER BY community
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df(),
    v_skills = v_skills
)

In [22]:
df_3.head(10)

,community,rank,email,person_name,skills
0,0,5.408388,joseph.lopez@test.org,Joseph Lopez,"[CI/CD, Django, Linux, ReactJS, System Design,..."
1,1,8.097379,david.rodriguez@test.org,David Rodriguez,"[Azure, Cypher, Scrum, Spark]"
2,1,5.411052,sophia.walker@test.org,Sophia Walker,"[C++, DevOps, Django, Scrum]"
3,2,5.408567,sophie.perez@test.org,Sophie Perez,"[Angular, Cloud Architecture, Scrum]"
4,2,5.408567,isabella.jones@test.org,Isabella Jones,"[Cloud Architecture, ReactJS, Scrum, TensorFlow]"
5,3,5.481543,isabella.allen@test.org,Isabella Allen,"[Cloud Architecture, Scala, Security]"
6,3,5.475172,olivia.johnson@test.org,Olivia Johnson,"[Angular, CI/CD, Security]"
7,3,5.411007,natalie.miller@test.org,Natalie Miller,"[Azure, Express.js, Go, Machine Learning, Test..."
8,4,5.417244,andrew.anderson@test.org,Andrew Anderson,"[Data Visualization, DevOps, Java, Node.js, Te..."
9,4,5.414759,andrew.martin@test.org,Andrew Martin,"[Cloud Architecture, Java, Node.js, R, Testing]"


# Lets create a Retrieval agent

In [23]:
class Skill(BaseModel):
    """
    Represents a professional skill or knwoledge of a person.
    """
    name: str = Field(..., description="Sortened name of the skill")

@tool
def retrieve_persons_given_set_of_skills(skills: List[Skill]) -> pd.DataFrame:
    """
    Given a list of skills this function will retrieve relevant persons. 
    If the question contains multiple skills, only call the function once using the set of
    skills as the argument for the function
    """
    skills = [s.name for s in skills]
    print(skills)
    v_skills = embeddings.embed_documents(skills)
    return driver.execute_query(
        """ 
        UNWIND $v_skills AS v
        CALL db.index.vector.queryNodes('ada_v', 10, TOFLOATLIST(v)) YIELD node
        WITH v, COLLECT(node)[0..3] AS top3
        UNWIND top3 AS s
        WITH COLLECT(s) AS skill_list 
        MATCH (p:Person)-[:KNOWS]->(s)
        WHERE s IN skill_list
        WITH p, SUM(REDUCE(res=0.0, x IN $v_skills | res + vector.similarity.cosine(x, s.embedding))) AS score
        WITH
            p.kmeans5_cluster AS community,
            score AS rank, 
            p.email AS email, 
            p.name AS person_name, 
            COLLECT{ MATCH (p)-[:KNOWS]->(anySkill) RETURN anySkill.name ORDER BY anySkill.name } AS skills
        ORDER BY rank DESC 
        LIMIT 10
        RETURN 
            community,
            rank,
            email,
            person_name,
            skills
        ORDER BY community
        """,
        database_=DATABASE,
        v_skills=v_skills,
        routing_=RoutingControl.READ,
        result_transformer_= lambda r: r.to_df()
    )

In [24]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [25]:
tools = [
    retrieve_persons_given_set_of_skills
]

In [26]:
llm_with_tools = llm.bind_tools(tools)

In [27]:
response = llm_with_tools.invoke("I am looking for a senior java developer that also knows cypher").tool_calls

In [28]:
response

[{'name': 'retrieve_persons_given_set_of_skills',
  'args': {'skills': [{'name': 'java'}, {'name': 'cypher'}]},
  'id': 'call_o5ICPBFvhas7uhJ8HOSoU7BZ',
  'type': 'tool_call'}]

In [29]:
name_to_functions = {
    'retrieve_persons_given_set_of_skills': retrieve_persons_given_set_of_skills,
}

In [30]:
selected_tool = name_to_functions[response[0]['name']]
args = response[0]['args']

In [31]:
df_response = selected_tool.invoke(args)

['java', 'cypher']


In [32]:
df_response.head(10)

,community,rank,email,person_name,skills
0,0.0,1.878038,charlotte.adams@test.org,Charlotte Adams,"[Blockchain, Java, Kubernetes, ReactJS]"
1,1.0,1.878038,david.hill@test.org,David Hill,"[Angular, Java, Scrum]"
2,2.0,1.878038,joseph.martin@test.org,Joseph Martin,"[Agile, Java, Linux, Power BI, ReactJS]"
3,4.0,1.878038,charles.taylor@test.org,Charles Taylor,"[CI/CD, Java, TensorFlow]"
4,4.0,1.878038,andrew.anderson@test.org,Andrew Anderson,"[Data Visualization, DevOps, Java, Node.js, Te..."
5,4.0,1.878038,thomas.garcia@test.org,Thomas Garcia,"[Agile, Docker, Flask, Java, TensorFlow]"
6,4.0,1.878038,amelia.davis@test.org,Amelia Davis,"[Docker, HTML5, Java, PyTorch, Security]"
7,4.0,1.878038,andrew.martin@test.org,Andrew Martin,"[Cloud Architecture, Java, Node.js, R, Testing]"
8,4.0,1.878038,thomas.brown@test.org,Thomas Brown,"[Docker, Java, R, Security]"
9,NaN,1.878038,hakan.lofqvist@neo4j.com,Håkan Löfqvist,"[Cypher Queries, Graph Technology, Java]"
